In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load Data

In [ ]:
train_data = pd.read_csv('../input/nlp-getting-started/train.csv')
test_data = pd.read_csv('../input/nlp-getting-started/test.csv')

print(f'train shape => {train_data.shape}')
print(f'test shape => {test_data.shape}')

# Examples data

In [ ]:
train_data.sample(5)

In [ ]:
test_data.sample(5)

# Describe

In [ ]:
train_data.describe(include=['O'])

In [ ]:
test_data.describe(include=['O'])

# Missing data

In [ ]:
def missing_values(data: pd.DataFrame) -> pd.DataFrame:
    miss_value_percent = data.isna().sum() / data.shape[0] * 100
    return pd.DataFrame(miss_value_percent, columns=['Missing_percent']).query('Missing_percent > 0')

In [ ]:
missing_values(train_data)

In [ ]:
missing_values(test_data)

# Distribution of the target variable

In [ ]:
sns.countplot(train_data.target);
plt.ylabel('Samples');

# Examples tweets

In [ ]:
train_data.query('target == 0').text.values[0]

In [ ]:
train_data.query('target == 1').text.values[0]

# Distribution of the target variable

In [ ]:
train_data.location.unique()

In [ ]:
train_data.location.nunique()

In [ ]:
sns.barplot(y=train_data['location'].value_counts()[:10].index,
            x=train_data['location'].value_counts()[:10]);

In [ ]:
train_data.text.values

# Preprocessing

In [ ]:
import re
import unidecode

In [ ]:
url = re.compile('https?://\S+|www\.\S+')

In [ ]:
url.sub('', 'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ')

In [ ]:
url.sub('', 'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ')

In [ ]:
import string

In [ ]:
def preprocessing(text):
    text = text.lower()
    text = unidecode.unidecode(text)
    
    text = re.sub('\[.*?\]', '', text)
    
    text_without_url = re.sub('https?://\S+|www\.\S+', '', text)
    
    text_without_tag = re.sub('<.*?>+', '', text_without_url)
    
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text_without_tag)
    
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    
    return text

In [ ]:
train_data = pd.read_csv('../input/nlp-getting-started/train.csv')
test_data = pd.read_csv('../input/nlp-getting-started/test.csv')

In [ ]:
train_data.text = train_data.text.apply(preprocessing)
test_data.text = test_data.text.apply(preprocessing)

In [ ]:
train_data.text

# Tokenization

In [ ]:
import nltk

In [ ]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

In [ ]:
train_data.text = train_data.text.apply(tokenizer.tokenize)
test_data.text = test_data.text.apply(tokenizer.tokenize)

In [ ]:
train_data

# Remove stopwords

In [ ]:
from nltk.corpus import stopwords

In [ ]:
train_data.text = train_data.text.apply(lambda text: [w for w in text if w not in stopwords.words('english')])
test_data.text = test_data.text.apply(lambda text: [w for w in text if w not in stopwords.words('english')])

In [ ]:
train_data.text = train_data.text.apply(lambda text: ' '.join(text))
test_data.text = test_data.text.apply(lambda text: ' '.join(text))

In [ ]:
train_data.text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data.text.values,
                                                    train_data.target.values,
                                                    stratify=train_data.target.values,
                                                    test_size=0.2, random_state=1)

In [ ]:
tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1, 2))
train_tfidf = tfidf.fit_transform(X_train)
test_tfidf = tfidf.transform(X_test)
test = tfidf.transform(test_data.text)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score

In [ ]:
# def grid_params_model(model, params, X_train, y_train, X_dev, y_dev):
# #     clf = Pipeline(steps=[('preprocessor', preprocessor),
# #                       ('model', model)
# #                      ])
#     print(clf.get_params().keys())
#     grid_search = GridSearchCV(clf, params, cv=5, scoring='f1')
#     grid_search.fit(X_train, y_train)

#     model_gs_best = grid_search.best_estimator_ 

#     metrics_name = ["Mean cross-validated score", "f1_score train", "f1_score dev"]
#     metrics_values = [grid_search.best_score_,
#                     f1_score(y_train, model_gs_best.predict(X_train)),
#                     f1_score(y_dev, model_gs_best.predict(X_dev))
#                      ]

#     metrics_res = {k : v for k, v in zip(metrics_name, metrics_values)}

#     return model_gs_best, metrics_res

In [ ]:
clf = LogisticRegression(C=1.0)
scores = cross_val_score(clf, train_tfidf, y_train, cv=5, scoring="f1")
scores

In [ ]:
clf.fit(train_tfidf, y_train)

In [ ]:
f1_score(y_test, clf.predict(test_tfidf))

In [ ]:
clf.predict(test)

In [ ]:
pd.read_csv('../input/nlp-getting-started/test.csv').id.values

In [ ]:
submission = pd.DataFrame({'id': pd.read_csv('../input/nlp-getting-started/test.csv').id.values,
                           'target': clf.predict(test)})

In [ ]:
submission.to_csv('submission.csv', index=False)